In [1]:
import os
os.chdir("/app")
from src.preprocessing import preprocess
from itertools import accumulate, chain
from omegaconf import OmegaConf
import torch
import seaborn as sns
from matplotlib import pyplot as plt
from collections import Counter
import pandas as pd
from ptls.preprocessing import PandasDataPreprocessor

In [2]:
train, val, test = preprocess(OmegaConf.load("config/preprocessing/churn_nodup.yaml"))

In [3]:
times = [row["event_time"] for row in train]

In [4]:
def get_subseqs_lens(seq: torch.Tensor):
    is_same_times = (seq.diff() == 0).int().tolist()
    subseqs_lens = []
    subseq_len = 1
    for is_same in is_same_times:
        if is_same:
            subseq_len += 1
        else:
            subseqs_lens.append(subseq_len)
            subseq_len = 1
            
    return subseqs_lens
    

In [5]:
subseqs_lens = [subseq_len for time in times for subseq_len in get_subseqs_lens(time)]

In [6]:
subseq_lens_counter = pd.Series(subseqs_lens).value_counts(normalize=True)

In [7]:
subseq_lens_counter

1    1.0
dtype: float64

In [51]:
subseq_lens_counter.iloc[0:].sum()

0.9999999999999999

In [24]:
num_days = sum(counter.values())
frac_same = {k: v / num_days for k, v in counter.items()}

In [25]:
frac_same

{1: 0.5325966436518097,
 2: 0.19488888055664172,
 3: 0.12076640009427342,
 4: 0.07006765189537792,
 5: 0.038453915828829455,
 7: 0.010546839429860035,
 6: 0.02008131082912406,
 15: 8.570311370124966e-05,
 13: 0.00026246578571007705,
 10: 0.0015105173789845253,
 12: 0.0005624266836644509,
 9: 0.0029139058658424885,
 8: 0.00579031661944068,
 11: 0.0010070115859896836,
 14: 0.00020890133964679604,
 20: 1.0712889212656207e-05,
 18: 3.749511224429673e-05,
 17: 2.1425778425312414e-05,
 19: 1.606933381898431e-05,
 22: 2.1425778425312414e-05,
 16: 5.892089066960914e-05,
 24: 1.606933381898431e-05,
 27: 5.356444606328104e-06,
 25: 1.0712889212656207e-05,
 29: 1.0712889212656207e-05,
 21: 2.1425778425312414e-05,
 30: 5.356444606328104e-06,
 23: 5.356444606328104e-06,
 45: 5.356444606328104e-06,
 38: 5.356444606328104e-06,
 31: 5.356444606328104e-06}